# Feature Selection and Filtering

_Abdurrahman Dilmac, Ugur Ali Kaplan_

12nd February 2022

In [1]:
import numpy as np
import pandas as pd

from imblearn.combine import SMOTETomek
from plotnine import ggplot, geom_point, aes, scale_color_cmap_d
from sklearn.decomposition import PCA
from sklearn.dummy import DummyClassifier
from sklearn.feature_selection import VarianceThreshold, SelectKBest, chi2, RFE
from sklearn.manifold import TSNE
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier

## Load and Split Data

You can download the Santander Customer Transaction Prediction data from here: [https://www.kaggle.com/c/santander-customer-transaction-prediction/data](https://www.kaggle.com/c/santander-customer-transaction-prediction/data) Since it has some terms of use, we won't be uploading it to the repository.

In [2]:
train_data = pd.read_csv("santander-customer-transaction-prediction/train.csv", index_col="ID_code")
#test_data = pd.read_csv("santander-customer-transaction-prediction/test.csv", index_col="ID_code")

We then pick only random 500 samples.

In [3]:
train_data = train_data.sample(500, random_state=0)

In [4]:
train_data.to_csv("santander-customer-transaction-prediction/train_subset.csv", index=True)

In [5]:
#train_data = pd.read_csv("santander-customer-transaction-prediction/train_subset.csv", index_col="ID_code")

We split the data with 20% to be reserved for test.

In [6]:
x_train, x_val, y_train, y_val = train_test_split(train_data.drop(["target"], axis=1), train_data["target"], test_size=0.2, random_state=0)

## Get Some Baselines

When we are doing a machine learning task, we always need some baseline, i.e. an acceptance criteria. On data hackathons, it is sometimes given or in a business, the goal is often pre-set. We can also generate some baselines using `sklearn.dummy.DummyClassifier`. If we cannot perform better than a dummy classifier, then there is no need to run calculations and harm the planet.

Let's define three dummy classifiers. Our criterias are:

1. `stratified`: Random
2. `uniform`: Each class has the same probability
3. `most_frequent`: The most frequent class has 100% probability.

In [7]:
stratified_base = DummyClassifier(strategy="stratified", random_state=0)
uniform_base = DummyClassifier(strategy="uniform", random_state=0)
frequent_base = DummyClassifier(strategy="most_frequent", random_state=0)

In [8]:
stratified_base.fit(x_train, y_train)
print(classification_report(y_val, stratified_base.predict(x_val)))
print(roc_auc_score(y_val, stratified_base.predict_proba(x_val)[:, 1]))

              precision    recall  f1-score   support

           0       0.90      0.90      0.90        91
           1       0.00      0.00      0.00         9

    accuracy                           0.82       100
   macro avg       0.45      0.45      0.45       100
weighted avg       0.82      0.82      0.82       100

0.45054945054945056


In [9]:
uniform_base.fit(x_train, y_train)
print(classification_report(y_val, uniform_base.predict(x_val)))
print(roc_auc_score(y_val, uniform_base.predict_proba(x_val)[:, 1]))

              precision    recall  f1-score   support

           0       0.91      0.44      0.59        91
           1       0.09      0.56      0.15         9

    accuracy                           0.45       100
   macro avg       0.50      0.50      0.37       100
weighted avg       0.84      0.45      0.55       100

0.5


In [10]:
import warnings # Some tuning ;) 
warnings.filterwarnings('ignore')

frequent_base.fit(x_train, y_train)
print(classification_report(y_val, frequent_base.predict(x_val)))
print(roc_auc_score(y_val, frequent_base.predict_proba(x_val)[:, 1]))

warnings.filterwarnings('default')

              precision    recall  f1-score   support

           0       0.91      1.00      0.95        91
           1       0.00      0.00      0.00         9

    accuracy                           0.91       100
   macro avg       0.46      0.50      0.48       100
weighted avg       0.83      0.91      0.87       100

0.5


In case you don't remember precision and/or recall, this is good time to remember.

<figure>
    <img src="img/precision_recall.png">
    <figcaption>Precision, recall, accuracy. <br> (Figure from: <i>https://towardsdatascience.com/precision-vs-recall-386cf9f89488</i>)</figcaption>
</figure>

## Investigate the Data

Let's get an overview of the data. This is quite helpful!

In [11]:
x_train.describe()

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
count,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,...,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000
mean,10.859090,-1.661859,10.844833,6.773471,11.155398,-5.305423,5.445151,16.747193,0.154987,7.584889,...,3.215160,7.502116,1.817218,3.264477,17.658186,0.010038,2.504868,8.891467,15.936220,-2.903208
std,2.975612,4.045273,2.618304,2.046323,1.661087,8.301437,0.866293,3.371595,3.394990,1.172439,...,4.610747,3.115134,1.432869,3.905974,3.068203,1.486291,5.264922,0.902115,2.978685,10.191365
min,3.126200,-11.073100,4.288400,1.129000,7.028500,-26.124700,3.024200,8.876100,-8.356900,4.656300,...,-7.287000,-0.163800,-2.305400,-7.691900,10.870600,-3.866900,-8.522300,6.343300,8.161800,-30.008600
25%,8.713850,-4.829725,8.799600,5.116200,9.883050,-11.928125,4.799875,14.331875,-2.470500,6.707925,...,0.005200,5.187925,0.841650,0.804450,15.302475,-1.020275,-1.396975,8.277850,13.834850,-10.519550
50%,10.670850,-1.638900,10.856800,6.837650,11.227250,-5.116750,5.481050,16.534050,0.054700,7.679650,...,3.304200,7.329950,1.847650,3.211650,17.513700,0.044350,2.530750,8.884150,15.847750,-2.650650
75%,12.832975,1.473550,12.616875,8.250875,12.374550,1.411725,6.064200,19.217700,3.012575,8.521800,...,6.432600,9.807475,2.786350,6.204425,20.083350,1.004550,6.908175,9.517375,18.104350,4.846150
max,18.852700,8.415000,17.325200,11.503700,14.754500,14.138600,7.718800,25.282000,8.521800,9.994000,...,14.657900,16.336500,5.412100,13.371200,25.932700,3.374300,14.604200,11.358900,23.414800,19.298000


Some datasets might contain `null` values. We check that.

In [12]:
x_train.isna().sum().sum()

0

We now describe train labels.

In [13]:
y_train.describe()

count    400.000000
mean       0.102500
std        0.303685
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max        1.000000
Name: target, dtype: float64

The labels determine if we need to use a regression or classification method. Here, we are doing classification and now let's see our unique labels.

In [14]:
y_train.unique()

array([1, 0])

We check to see if our labels are balanced or not.

In [15]:
(y_train == 1).sum() / len(y_train)

0.1025

In [16]:
(y_train == 0).sum() / len(y_train)

0.8975

Now we have encountered an imbalance problem in our labels. We have to mitigate to get better results.

## Imbalance Problem

We will fix imbalance problem by using SMOTE resampler. Note that our resampler might require an intensive load of memory.

In [17]:
resampler = SMOTETomek(random_state=0)

In [18]:
x_train, y_train = resampler.fit_resample(x_train, y_train)

Let's save our valuable data. 

In [19]:
x_train.to_csv("santander-customer-transaction-prediction/smote_train.csv", index=False)
y_train.to_csv("santander-customer-transaction-prediction/smote_train_labels.csv", index=False)
x_val.to_csv("santander-customer-transaction-prediction/X_val.csv")
y_val.to_csv("santander-customer-transaction-prediction/y_val.csv")

In [20]:
x_train = pd.read_csv("santander-customer-transaction-prediction/smote_train.csv")
y_train = pd.read_csv("santander-customer-transaction-prediction/smote_train_labels.csv")

Now let's check if our labels are balanced.

In [21]:
(y_train == 1).sum() / len(y_train)

target    0.5
dtype: float64

In [22]:
(y_train == 0).sum() / len(y_train)

target    0.5
dtype: float64

Finally let's see how many data we have now.

In [23]:
len(x_train)

718

As we can see, we have increased the number of data points by doing over-sampling and under-sampling.

## Feature Extraction and Selection

After some practice, we get ourselves convinced that we don't always need too many features and to keep scarce resources, we want to take the most important features.

### 1. PCA

First we will try Principal Component Analysis (PCA) for our data. We want the PCA library to infer the number of components, thus we give its argument as `mle`.

In [24]:
pca = PCA(n_components="mle")

In [25]:
x_train_pca = pca.fit_transform(x_train, y_train)

In [26]:
x_train_pca.shape

(718, 199)

It seems PCA doesn't work that well. One reason might be that PCA is already applied to our data or we are supplied a parametrically randomized data.

Now we have seen PCA doesn't work well with our data, we remove it from the memory and won't use.

In [27]:
del(x_train_pca)

### 2. Get rid of the features having low variance

We use `sklearn.feature_selection.VarianceThreshold` for such purpose. We set a threshold and fit the train data. It removes all the data having lower variances than our threshold.

In [28]:
selector = VarianceThreshold(threshold=0.16)

In [29]:
selector.fit(x_train)

VarianceThreshold(threshold=0.16)

In [30]:
x_train = x_train.T[selector.get_support()].T

In [31]:
x_train.describe()

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
count,718.000000,718.000000,718.000000,718.000000,718.000000,718.000000,718.000000,718.000000,718.000000,718.000000,...,718.000000,718.000000,718.000000,718.000000,718.000000,718.000000,718.000000,718.000000,718.000000,718.000000
mean,10.721856,-1.691721,11.063598,6.891730,11.284924,-5.373452,5.495469,16.799569,-0.308292,7.430141,...,3.924898,7.903530,1.703081,3.111653,17.204733,0.052557,2.285628,8.805779,15.808148,-2.569427
std,2.730058,3.690532,2.484651,1.758038,1.528063,7.639730,0.842615,2.983098,3.194224,1.069603,...,4.440823,2.977094,1.283173,3.598167,2.748902,1.328581,5.080933,0.804088,2.933997,9.367677
min,3.126200,-11.073100,4.288400,1.129000,7.028500,-26.124700,3.024200,8.876100,-8.356900,4.656300,...,-7.287000,-0.163800,-2.305400,-7.691900,10.870600,-3.866900,-8.522300,6.343300,8.161800,-30.008600
25%,8.639775,-4.307950,9.222050,5.697600,10.198044,-11.289461,4.884600,14.670746,-2.875301,6.635428,...,1.006375,5.494373,0.863424,1.024475,15.279588,-0.860427,-1.155666,8.280334,13.548000,-9.485537
50%,10.419850,-1.703750,10.983700,6.967602,11.313478,-5.147943,5.506400,16.779360,-0.565750,7.486048,...,4.107604,7.798550,1.637946,3.135381,17.021300,0.044229,2.389153,8.768800,15.902467,-2.472670
75%,12.531878,0.871600,12.773288,8.073950,12.502467,0.137795,6.127232,18.802669,2.115958,8.195158,...,7.321282,9.992392,2.572880,5.624264,18.990207,0.925700,6.083484,9.319899,17.910597,4.741587
max,18.852700,8.415000,17.325200,11.503700,14.754500,14.138600,7.718800,25.282000,8.521800,9.994000,...,14.657900,16.336500,5.412100,13.371200,25.932700,3.374300,14.604200,11.358900,23.414800,19.298000


3. Scaling

For most of the classifiers, it is very important that our features share the same scale. It might significantly improve our metrics. For this purpose, we use `sklearn.preprocessing.MinMaxScaler`.

In [32]:
scaler = MinMaxScaler()

In [33]:
scaler.fit(x_train)

MinMaxScaler()

In [34]:
x_train = pd.DataFrame(scaler.transform(x_train), columns=x_train.columns)

In [35]:
x_train.describe()

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
count,718.000000,718.000000,718.000000,718.000000,718.000000,718.000000,718.000000,718.000000,718.000000,718.000000,...,718.000000,718.000000,718.000000,718.000000,718.000000,718.000000,718.000000,718.000000,718.000000,718.000000
mean,0.482985,0.481390,0.519698,0.555460,0.550922,0.515389,0.526407,0.482965,0.476850,0.519670,...,0.510911,0.488920,0.519401,0.512914,0.420535,0.541272,0.467340,0.490964,0.501301,0.556501
std,0.173596,0.189374,0.190587,0.169454,0.197782,0.189744,0.179486,0.181831,0.189246,0.200386,...,0.202362,0.180427,0.166268,0.170828,0.182505,0.183475,0.219702,0.160317,0.192355,0.189988
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.350591,0.347143,0.378440,0.440360,0.410244,0.368456,0.396285,0.353205,0.324764,0.370783,...,0.377918,0.342913,0.410602,0.413822,0.292721,0.415190,0.318536,0.386202,0.353124,0.416234
50%,0.463781,0.480773,0.513569,0.562773,0.554618,0.520989,0.528735,0.481733,0.461597,0.530144,...,0.519237,0.482558,0.510962,0.514040,0.408356,0.540122,0.471816,0.483591,0.507485,0.558463
75%,0.598078,0.612923,0.650841,0.669412,0.708512,0.652269,0.660979,0.605061,0.620478,0.662993,...,0.665680,0.615516,0.632106,0.632203,0.539075,0.661852,0.631561,0.593468,0.639140,0.704778
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### 4. Feature Selection

Now we can try different feature selection methods.

#### 4.1. Select K-Best

We can select K-Best features by supplying a feature selection metric. Often, Chi-Squared stats between each feature is used. There are other metrics at our use, which can be found [here](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html).

In [36]:
kbest = SelectKBest(chi2, k=60)

In [37]:
kbest.fit_transform(x_train, y_train)

array([[0.30979228, 0.60772716, 0.276686  , ..., 0.28801295, 0.51344102,
        0.5482694 ],
       [0.24553571, 0.79597463, 0.01253651, ..., 0.36425398, 0.10249567,
        0.38123056],
       [0.52324957, 0.37674087, 0.45273629, ..., 0.18657237, 0.3202807 ,
        0.53343568],
       ...,
       [0.32678378, 0.47153694, 0.28465091, ..., 0.68253543, 0.53789667,
        0.46257938],
       [0.31647097, 0.8280726 , 0.54158392, ..., 0.60651719, 0.30561803,
        0.39299503],
       [0.51637016, 0.71230168, 0.68529588, ..., 0.73209657, 0.41478585,
        0.46799057]])

In [38]:
x_kbest = x_train.T[kbest.get_support()].T

In [39]:
x_kbest

,var_2,var_4,var_8,var_9,var_13,var_14,var_15,var_21,var_22,var_24,...,var_165,var_176,var_181,var_182,var_184,var_188,var_190,var_191,var_194,var_197
0,0.309792,0.607727,0.276686,0.482998,0.345844,0.576175,0.753800,0.348432,0.482746,0.435809,...,0.294083,0.715781,0.335653,0.458027,0.610328,0.284849,0.413426,0.288013,0.513441,0.548269
1,0.245536,0.795975,0.012537,0.385128,0.563401,0.262186,0.585022,0.205552,0.401852,0.573300,...,0.193782,0.512708,0.461915,0.725640,0.474049,0.125588,0.368322,0.364254,0.102496,0.381231
2,0.523250,0.376741,0.452736,0.345111,0.579171,0.823897,0.435539,0.660887,0.477380,0.183767,...,0.417127,0.481505,0.059230,0.000000,0.591476,0.592808,0.369398,0.186572,0.320281,0.533436
3,0.340245,0.731931,0.290561,0.582030,0.254131,0.662745,0.241084,0.315342,0.430467,0.423553,...,0.247713,0.409862,0.535203,0.820549,0.225386,0.869040,0.341514,0.495924,0.169385,0.469814
4,0.314095,0.659293,0.618655,0.487214,1.000000,0.267410,0.592623,0.732848,0.507633,0.740072,...,0.873551,0.407762,0.457163,0.668456,0.700009,0.579469,0.685485,0.635055,0.225247,0.346439
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
713,0.356426,0.723965,0.422514,0.176090,0.636948,0.501959,0.646953,0.327674,0.299374,0.260968,...,0.514495,0.534583,0.641467,0.350326,0.610312,0.486795,0.281765,0.198378,0.259282,0.264989
714,0.441397,0.805958,0.536954,0.623706,0.178773,0.573111,0.690982,0.574916,0.415317,0.371026,...,0.545751,0.585062,0.542773,0.265673,0.686484,0.316970,0.628348,0.621138,0.364678,0.445926
715,0.326784,0.471537,0.284651,0.607745,0.213396,0.277463,0.719568,0.378004,0.223142,0.345993,...,0.579102,0.598857,0.575475,0.384488,0.904544,0.186520,0.720830,0.682535,0.537897,0.462579
716,0.316471,0.828073,0.541584,0.429601,0.381370,0.292233,0.194252,0.417400,0.450939,0.318285,...,0.407925,0.534456,0.753053,0.348014,0.654154,0.281801,0.702470,0.606517,0.305618,0.392995


### 4.2. Selection by Elimination

We use Recursive Feature Elimination (RFE) and eliminate features step by step. To do so, first, we supply a classifier and the selector tries it. Then the least important features are pruned. This process is repeated till we reach the number of features.

In [40]:
selector = RFE(DecisionTreeClassifier(random_state=0), n_features_to_select=60, step=5)

In [41]:
selector.fit(x_train, y_train)

RFE(estimator=DecisionTreeClassifier(random_state=0), n_features_to_select=60,
    step=5)

In [42]:
x_selected = x_train.T[selector.get_support()].T

In [43]:
x_selected

,var_10,var_11,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,...,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_194
0,0.411383,0.703717,0.345844,0.576175,0.753800,0.614011,0.682872,0.291305,0.774213,0.328036,...,0.335653,0.458027,0.094038,0.610328,0.483108,0.914478,1.000000,0.284849,0.108508,0.513441
1,0.503423,0.826755,0.563401,0.262186,0.585022,0.342418,0.514921,0.696600,0.870895,0.256716,...,0.461915,0.725640,0.559792,0.474049,0.592013,0.492822,0.472784,0.125588,0.279725,0.102496
2,0.768946,0.348468,0.579171,0.823897,0.435539,0.629032,0.165316,0.700003,0.584782,0.384021,...,0.059230,0.000000,0.826327,0.591476,0.463101,0.291125,0.181545,0.592808,0.524294,0.320281
3,0.459007,0.710080,0.254131,0.662745,0.241084,0.039496,0.387021,0.449781,0.462610,0.862184,...,0.535203,0.820549,0.650334,0.225386,0.528365,0.388445,0.558091,0.869040,0.540977,0.169385
4,0.862512,0.426559,1.000000,0.267410,0.592623,0.520469,0.412520,0.184917,0.620453,0.241272,...,0.457163,0.668456,0.495360,0.700009,0.386856,0.447948,0.677761,0.579469,0.565532,0.225247
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
713,0.797328,0.863336,0.636948,0.501959,0.646953,0.560428,0.625473,0.458281,0.636042,0.519508,...,0.641467,0.350326,0.620183,0.610312,0.439190,0.484970,0.454183,0.486795,0.675640,0.259282
714,0.407688,0.616841,0.178773,0.573111,0.690982,0.591048,0.663786,0.419126,0.664257,0.600287,...,0.542773,0.265673,0.306860,0.686484,0.422831,0.465041,0.439636,0.316970,0.475590,0.364678
715,0.568264,0.585716,0.213396,0.277463,0.719568,0.564902,0.836030,0.315599,0.762410,0.600242,...,0.575475,0.384488,0.530316,0.904544,0.462598,0.599895,0.511603,0.186520,0.389914,0.537897
716,0.724023,0.697342,0.381370,0.292233,0.194252,0.596121,0.758795,0.577215,0.829595,0.390092,...,0.753053,0.348014,0.363583,0.654154,0.633280,0.578690,0.350550,0.281801,0.579145,0.305618


### Comparison

Let's compare the difference in features that Select K-Best and Selection by Elimination methods choose.

In [44]:
set(x_kbest.columns) - set(x_selected.columns)

{'var_100',
 'var_112',
 'var_120',
 'var_122',
 'var_123',
 'var_124',
 'var_126',
 'var_137',
 'var_142',
 'var_144',
 'var_150',
 'var_154',
 'var_157',
 'var_160',
 'var_165',
 'var_190',
 'var_191',
 'var_197',
 'var_2',
 'var_38',
 'var_4',
 'var_41',
 'var_46',
 'var_49',
 'var_52',
 'var_55',
 'var_61',
 'var_67',
 'var_8',
 'var_9',
 'var_90',
 'var_96',
 'var_98',
 'var_99'}

In [45]:
set(x_selected.columns) - set(x_kbest.columns)

{'var_10',
 'var_11',
 'var_116',
 'var_117',
 'var_143',
 'var_155',
 'var_16',
 'var_168',
 'var_17',
 'var_172',
 'var_178',
 'var_18',
 'var_180',
 'var_183',
 'var_185',
 'var_186',
 'var_187',
 'var_189',
 'var_19',
 'var_20',
 'var_23',
 'var_26',
 'var_27',
 'var_28',
 'var_29',
 'var_30',
 'var_53',
 'var_56',
 'var_58',
 'var_59',
 'var_60',
 'var_63',
 'var_70',
 'var_75'}

### Save Column Names

It can be helpful to save and report our chosen features.

In [46]:
with open('santander-customer-transaction-prediction/kbest_column_names.txt', 'w') as f:
    f.writelines('\n'.join(list(x_kbest.columns)))

In [47]:
with open('santander-customer-transaction-prediction/selected_column_names.txt', 'w') as f:
    f.writelines('\n'.join(list(x_selected.columns)))